In [40]:
import os

In [41]:
%pwd

'c:\\Users\\Anuj Bohra\\Desktop\\MLOps'

In [42]:
os.chdir('c:\\Users\\Anuj Bohra\\Desktop\\MLOps')

In [43]:
%pwd

'c:\\Users\\Anuj Bohra\\Desktop\\MLOps'

In [44]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [45]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
from src.CNN_Classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH  # Import the paths

In [46]:
# Ensure CONFIG_FILE_PATH and PARAMS_FILE_PATH are correct
print("Root directory:", ROOT_DIR)
print("Config file path:", CONFIG_FILE_PATH)
print("Params file path:", PARAMS_FILE_PATH)
print("Config file exists:", CONFIG_FILE_PATH.exists())
print("Params file exists:", PARAMS_FILE_PATH.exists())


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH.resolve(),
                 params_filepath = PARAMS_FILE_PATH.resolve()):
        
                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)
                 
                 create_directories([self.config.artifacts_root])
                 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config
        

Root directory: C:\Users\Anuj Bohra\Desktop\MLOps\src\CNN_Classifier
Config file path: C:\Users\Anuj Bohra\Desktop\MLOps\config\config.yaml
Params file path: C:\Users\Anuj Bohra\Desktop\MLOps\params.yaml
Config file exists: True
Params file exists: True


In [47]:
import os
import urllib.request as request
import zipfile
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

[2024-12-03 11:08:02,412: INFO: common: yaml file: C:\Users\Anuj Bohra\Desktop\MLOps\config\config.yaml loaded successfully]
[2024-12-03 11:08:02,423: INFO: common: yaml file: C:\Users\Anuj Bohra\Desktop\MLOps\params.yaml loaded successfully]
[2024-12-03 11:08:02,427: INFO: common: created directory at: artifacts]
[2024-12-03 11:08:02,432: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-03 11:08:05,362: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 13EE:F5EFC:11D928:1553B4:674E9936
Accept-Ranges: bytes
Date: Tue, 03 Dec 2024 05:3